#model

In [ ]:
import pandas as pd

In [46]:
chattogram_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Chattogram.csv")
dhaka_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Dhaka.csv")
khulna_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Khulna.csv")
rajshahi_df = pd.read_csv("https://raw.githubusercontent.com/DryBoss/diarrhea-prediction-model/main/data/Rajshahi.csv")

In [48]:
# List of dataframes to process
dataframes = [chattogram_df, dhaka_df, khulna_df, rajshahi_df]

# Process each dataframe
for df in dataframes:
    # Ensure 'Date' is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Smooth each column (excluding 'Date') using a 30-day moving average
    for column in df.columns:
        if column != 'Date':  # Skip the 'Date' column
            smoothed_column = f'{column}_smooth'
            df[smoothed_column] = df[column].rolling(window=30).mean()


In [56]:
chattogram_df = chattogram_df.dropna()
dhaka_df = dhaka_df.dropna()
khulna_df = khulna_df.dropna()
rajshahi_df = rajshahi_df.dropna()

In [68]:
from sklearn.model_selection import train_test_split

X = chattogram_df.drop(columns=['Date', 'Diarrhea', 'Minimum Temperature',	'Maximum Temperature', 'Humidity', 'Preceptation', 'Diarrhea_smooth'])
y = chattogram_df['Diarrhea']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_regressor.fit(X_train, y_train)

# Make predictions
y_pred = rf_regressor.predict(X_test)


In [70]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

r2 = rf_regressor.score(X_test, y_test)
print("R² Score:", r2)


Mean Squared Error: 278.38621905298055
R² Score: 0.6789067814990957
